In [1]:
from gworacle import query, mass, snr
from astropy.utils.data import download_file
import json, numpy
from pycbc import init_logging

data = query.get_gracedb_superevents(cache=True)

# Downselect event gracedb
far_threshold = 1e-7 #2.0 / (86400 * 365.25) # This will exclude a lot of real BBH signals, but only the quiet ones

prod = [k for k in data if k['category'] == 'Production' and ('DQOK' in k['labels'] or 'ADVOK' in k['labels'])]
print("Production events", len(prod))

sig = [k for k in prod if k['far'] <= far_threshold]
print("Pass FAR threshold", len(sig))

ret = []
for s in sig:
    root_url = s['links']['files']
    fdir = download_file(root_url, cache=True)
    flist = json.load(open(fdir, 'r'))
    if 'Retraction' in ' '.join(flist):
        ret.append(s)

# Remove the retracted ones
for s in ret:
    sig.remove(s)
print("After removing retractions", len(sig))

# Organize by id
events = {k['superevent_id']: {'data':k} for k in sig}

/home/ahnitz/projects/env/lib64/python3.11/site-packages/pycbc/types/array.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal as _lal


Production events 4021
Pass FAR threshold 303
After removing retractions 266


In [14]:
from gworacle.sky import get_gracedb_skymap
from astropy.utils.data import download_file
from astropy.table import QTable
import astropy_healpix
from scipy.stats import norm

# Get the chirp masses when we can from the source classification
i = 1
rdata = {}
for event in events:
    time = events[event]['data']['t_0']
    ifos = events[event]['data']['preferred_event_data']['instruments'].split(',')
    se = events[event]
    sed = se['data']
    
    se['mchirp_class'] = mass.mchirp_from_source_class(sed)['mchirp']
    b = snr.get_bayestar_snr(sed)
    se['bsn'] = b['bsn']
    se['snr'] = b['snr']
    se['snr_err'] = b['snr_err']
    
    if se['mchirp_class'] is not None:
        print(i, event, se['mchirp_class'], se['bsn'], se['snr'], se['snr_err'])
        i += 1
    else:
        continue

    # try multiple and then compare
    skymaps = {}
    for type in ['bayestar', 'PublicationSamples', 'bilby', 'lalinference']:
        try:
            mname = get_gracedb_skymap(event, map_type=type, use_cache=True)
            skymap = QTable.read(mname)

            if 'PROBDENSITY' in skymap.dtype.names:
                level, ipix = astropy_healpix.uniq_to_level_ipix(skymap['UNIQ'])
                nside = astropy_healpix.level_to_nside(level)
                ra, dec = astropy_healpix.healpix_to_lonlat(ipix, nside, order='nested')
                pixel_area = astropy_healpix.nside_to_pixel_area(nside)
                probd = skymap['PROBDENSITY']
            elif 'PROB' in skymap.dtype.names:
                nside = int((len(skymap['PROB'])/ 12)**(1/2))
                ra, dec = astropy_healpix.healpix_to_lonlat(numpy.arange(0, len(skymap['PROB'])), nside)
                pixel_area = astropy_healpix.nside_to_pixel_area(nside)
                probd = skymap['PROB']  

            pix = probd.argmax() 
            dr = .1
            r = numpy.arange(0, 16000, dr)
            d = r**2.0  * skymap['DISTNORM'][pix] * norm(skymap['DISTMU'][pix],
                                                         skymap['DISTSIGMA'][pix]).pdf(r) * dr
            dist = r[d.argmax()]

            skymaps[type] = skymap, dist, ra, dec, pix
        except Exception as e:
            pass

    rdata[event] = {'dist':dist, 'mest': se['mchirp_class'],
                    'distmu': skymap['DISTMU'][pix].value, 
                    'distsigma': skymap['DISTSIGMA'][pix].value,
                    'distnorm': skymap['DISTNORM'][pix].value,
                    'snr': b, 'time': time,
                     'far': events[event]['data']['far'],
                     'ifos':ifos}    
    print(rdata[event])

import json
json.dump(rdata, open('class_massest.json', 'w'))

1 S250328ae 8.023674320375923 206.5341106566025 26.329942155559394 0.9943497835793864
{'dist': 523.4, 'mest': 8.023674320375923, 'distmu': 502.0964154275276, 'distsigma': 74.59959442062423, 'distnorm': 3.880994581290575e-06, 'snr': {'bsn': 206.5341106566025, 'snr': 26.329942155559394, 'snr_err': 0.9943497835793864}, 'time': 1427175645.419434, 'far': 3.167545842243851e-10, 'ifos': ['H1', 'L1', 'V1']}
2 S250211aa 8.719070542918693 27.159071218776468 11.881065119683935 0.6154389031165534
{'dist': 1171.5, 'mest': 8.719070542918693, 'distmu': 1045.487051787744, 'distsigma': 271.7431780349709, 'distnorm': 8.569809567482982e-07, 'snr': {'bsn': 27.159071218776468, 'snr': 11.881065119683935, 'snr_err': 0.6154389031165534}, 'time': 1423275964.623047, 'far': 4.675193283659444e-18, 'ifos': ['H1', 'L1', 'V1']}
3 S250206dm 1.8419061097365435 18.789048402496498 10.497984904882577 0.5791686697524037
{'dist': 389.70000000000005, 'mest': 1.8419061097365435, 'distmu': 340.9468611338439, 'distsigma': 97.4